In [13]:
from itertools import product
import itertools
import operator
import sys
import os
sys.path.append("../src")
from glob import glob

In [2]:
search_root = "/checkpoint/jth/job_staging/neuraldb_expts/experiment=lr_sweep"
checkpoint_name = "best_em.ckpt"
files = glob("{}*/**/{}".format(search_root,checkpoint_name), recursive=True)


print(len(files))

325


In [10]:
def expand(idx,chunk):
  if idx == 0:
    return ["job_id={}".format(chunk)]
  #elif idx == 1:
  #  return ["experiment={}".format(chunk)]
  elif chunk.startswith("seed-"):
    return ["seed={}".format(chunk.replace("seed-",""))]
  elif "," in chunk:
    return chunk.split(",")
  elif "=" in chunk:
    return [chunk]
  return []

experiments = []
for file in files:
    chunks = file.replace(search_root,"").split("/")
    chunks = itertools.chain(*[expand(idx, chunk) for idx, chunk in enumerate(chunks)])

    data = {k:v for k,v in (chunk.split("=") for chunk in chunks)}
    data["file"] = file
    data['dir'] = os.path.dirname(file)

    data["lr"] = float(data["lr"])
    data["context"] = int(data["context"])
    if data["version"]=="v0.4" and data["lr"]==1e-3 and (data['context'] == 1000 or data["context"]==100):
        print(data)
        experiments.append(data)

{'job_id': '', 'dataset': 'ndb', 'model': 't5-base', 'version': 'v0.4', 'lr': 0.001, 'context': 100, 'filters': 'null', 'max_queries': 'null', 'seed': '3', 'file': '/checkpoint/jth/job_staging/neuraldb_expts/experiment=lr_sweep/dataset=ndb,model=t5-base,version=v0.4/lr=1e-3/context=100,filters=null,max_queries=null/seed-3/best_em.ckpt', 'dir': '/checkpoint/jth/job_staging/neuraldb_expts/experiment=lr_sweep/dataset=ndb,model=t5-base,version=v0.4/lr=1e-3/context=100,filters=null,max_queries=null/seed-3'}
{'job_id': '', 'dataset': 'ndb', 'model': 't5-base', 'version': 'v0.4', 'lr': 0.001, 'context': 100, 'filters': 'null', 'max_queries': 'null', 'seed': '1', 'file': '/checkpoint/jth/job_staging/neuraldb_expts/experiment=lr_sweep/dataset=ndb,model=t5-base,version=v0.4/lr=1e-3/context=100,filters=null,max_queries=null/seed-1/best_em.ckpt', 'dir': '/checkpoint/jth/job_staging/neuraldb_expts/experiment=lr_sweep/dataset=ndb,model=t5-base,version=v0.4/lr=1e-3/context=100,filters=null,max_querie

In [27]:
modes = ["distractors","similar"]
size = [1,10,50,100,1000,2500,5000,7500,10000]
seeds = list(range(1,10+1))
val = list(product(modes,size,seeds))

In [29]:
expts = []
for exp in experiments:
    version = exp["version"]

    for ds, num, seed in val:
        expts.append("python -m neuraldb.commands.t5train2 "
              "--output_dir {} "
              "--model_name_or_path t5-base "
              "--do_predict "
              "--train_batch_size 8 "
              "--eval_batch_size 8 "
              "--n_gpu 2 "
              "--train_path={}/train_queries_last_{}.json "
              "--val_path={}/dev_queries_last_{}.json "
              "--test_path=generate_distrators_{}/dev_{}_{}_{}_{}.json "
              "--test_name=metrics_test_distractors_{}_{}_{} "
              "--dataset_version={}".format(exp['dir'],version,exp['context'],version,exp['context'],version,exp['context'],ds,num,seed,ds,num,seed,version))


print(len(expts))
for expt in expts:
    print(expt)

1800
python -m neuraldb.commands.t5train2 --output_dir /checkpoint/jth/job_staging/neuraldb_expts/experiment=lr_sweep/dataset=ndb,model=t5-base,version=v0.4/lr=1e-3/context=100,filters=null,max_queries=null/seed-3 --model_name_or_path t5-base --do_predict --train_batch_size 8 --eval_batch_size 8 --n_gpu 2 --train_path=v0.4/train_queries_last_100.json --val_path=v0.4/dev_queries_last_100.json --test_path=generate_distrators_v0.4/dev_100_distractors_1_1.json --test_name=metrics_test_distractors_distractors_1_1 --dataset_version=v0.4
python -m neuraldb.commands.t5train2 --output_dir /checkpoint/jth/job_staging/neuraldb_expts/experiment=lr_sweep/dataset=ndb,model=t5-base,version=v0.4/lr=1e-3/context=100,filters=null,max_queries=null/seed-3 --model_name_or_path t5-base --do_predict --train_batch_size 8 --eval_batch_size 8 --n_gpu 2 --train_path=v0.4/train_queries_last_100.json --val_path=v0.4/dev_queries_last_100.json --test_path=generate_distrators_v0.4/dev_100_distractors_1_2.json --test_